# 移动平均(moving average，MA, 移动平均线, 均线)
> 技术分析中一种分析时间序列数据的工具。最常见的是利用股价、回报或交易量等变数计算出移动平均。移动平均可抚平短期波动，反映出长期趋势或周期。数学上，移动平均可视为一种卷积。

In [1]:
import pandas as pd
stock_data = pd.read_csv('Data/stock data/sh600000.csv', parse_dates=[1])

# 将数据按照交易日期从小到大排序
stock_data.sort_values('date', inplace=True)
stock_data[:5]

,code,date,open,high,low,close,change,volume,money,traded_market_value,market_value,turnover,adjust_price,report_type,report_date,PE_TTM,PS_TTM,PC_TTM,PB,adjust_price_f
481,sh600000,2013-01-04,10.10,10.26,9.93,10.02,0.010081,261436810.0,2.633797e+09,1.495262e+11,1.869078e+11,0.017519,73.420510,2012-09-30,2012-10-31,5.579037,2.346218,0.985053,1.101442,8.500869
480,sh600000,2013-01-07,10.05,10.34,10.01,10.32,0.029940,248200915.0,2.534162e+09,1.540031e+11,1.925038e+11,0.016632,75.618720,2012-09-30,2012-10-31,5.746073,2.416463,1.014545,1.134419,8.755386
479,sh600000,2013-01-08,10.34,10.35,10.06,10.12,-0.019380,154851189.0,1.573240e+09,1.510185e+11,1.887731e+11,0.010377,74.153229,2012-09-30,2012-10-31,5.634714,2.369632,0.994883,1.112434,8.585706
478,sh600000,2013-01-09,10.09,10.18,9.98,10.14,0.001976,175906181.0,1.774560e+09,1.513170e+11,1.891462e+11,0.011788,74.299756,2012-09-30,2012-10-31,5.645848,2.374315,0.996849,1.114632,8.602671
477,sh600000,2013-01-10,10.11,10.21,9.99,10.04,-0.009862,121226794.0,1.224889e+09,1.498247e+11,1.872809e+11,0.008124,73.567011,2012-09-30,2012-10-31,5.590169,2.350899,0.987018,1.103639,8.517832


### 简单移动平均(simple moving average，SMA)
> 是某变数之前n个数值的未作加权算术平均。例如，收市价的10日简单移动平均指之前10日收市价的平均数。

若设收市价为$p_1$至$p_n$，则方程式为：
$$SMA = \frac{p_1 + p_2 + \cdot\cdot\cdot +p_n}{n}$$

当计算连续的数值，一个新的数值加入，同时一个旧数值剔出，所以无需每次都重新逐个数值加起来：
$$SMA_{t1,n} = SMA_{t0,n} - \frac{p_1}{n} + \frac{p_{n+1}}{n}$$

在技术分析中，不同的市场对常用天数（n值）有不同的需求，
例如：某些市场普遍的n值为10日、40日、200日；有些则是5日、10日、20日、60日、120日、240日，视乎分析时期长短而定。
投资者冀从移动平均线的图表中分辨出支持位或阻力位。

In [2]:
# 计算5日SMA
# stock_data['close']为股票每天的收盘价
stock_data['MA_5'] = stock_data['close'].rolling(window=5).mean()

# 查看前15数据的SMA, 因为SMA的n为5, 所以前四条没有数据
stock_data[['close','MA_5']][:15]

,close,MA_5
481,10.02,NaN
480,10.32,NaN
479,10.12,NaN
478,10.14,NaN
477,10.04,10.128
476,9.82,10.088
475,10.35,10.094
474,10.33,10.136
473,10.12,10.132
472,10.03,10.130


### 加权移动平均(weighted moving average, WMA)
> 计算平均值时将个别数据乘以不同数值

在技术分析中，n日WMA的最近期一个数值乘以n、次近的乘以n-1，如此类推，一直到0：

若设收市价为$p_1$至$p_n$，则方程式为：
$$WMA_M = \frac{np_M + (n-1)p_{M-1} + \cdot \cdot \cdot + 2p_{M-n} + p_{M-n+1}}{n+(n-1)+\cdot\cdot\cdot+2+1}$$

实现: https://stackoverflow.com/questions/9621362/how-do-i-compute-a-weighted-moving-average-using-pandas

### 指数移动平均(exponential moving average，EMA或EXMA)
> 以指数式递减加权的移动平均。各数值的加权影响力随时间而指数式递减，越近期的数据加权影响力越重，但较旧的数据也给予一定的加权值

加权的程度以常数α决定，α数值介乎0至1。α也可用天数N来代表：$\alpha ={2 \over {N+1}}$
设时间t的实际数值为Yt，而时间t的EMA则为$EMA_t$；时间t-1的EMA则为$EMA_{t-1}$，计算时间t≥2是方程式为
$$EMA_{t} = \begin{cases} Y_1, t=1 \\ \alpha Y_{t}+(1-\alpha ) EMA_{t-1}, t>1 \end{cases}$$

$\alpha$: 表示权重下降的程度
$Y_t$: 时间段t的值
$EMA_t$: 时间段t的EMA值

$EMA_1$可以通过多种不同的方式初始化，最常见的是通过如上所述将$EMA_1$设置为$Y_1$，尽管也存在其他技术，例如将$EMA_1$设置为前4或5个观测值的平均值。$EMA_1$初始化效应对所得移动平均值的重要性取决于$\alpha$；较小的$\alpha$值使$EMA_1$的选择比较大的$\alpha$值相对重要，因为较高的$\alpha$会使较早的观测值更快地折价。

In [5]:
# 计算指数平滑移动平均线EMA
# alpha = 2/(span+1)
# alpha = 1/(1+com)
# stock_data['EMA_5'] = stock_data['close'].ewm(span=5).mean()
# stock_data['EMA_5'] = stock_data['close'].ewm(alpha=1/3).mean()
stock_data['EMA_5'] = stock_data['close'].ewm(com=2).mean()
stock_data[['close','EMA_5']][:15]

,close,EMA_5
481,10.02,10.020000
480,10.32,10.200000
479,10.12,10.162105
478,10.14,10.152923
477,10.04,10.109573
476,9.82,10.003759
475,10.35,10.126348
474,10.33,10.196988
473,10.12,10.170640
472,10.03,10.122933


In [6]:
# 将数据按照交易日期从大到小排序
stock_data.sort_values('date', ascending=False, inplace=True)
stock_data[:5]

,code,date,open,high,low,close,change,volume,money,traded_market_value,...,adjust_price,report_type,report_date,PE_TTM,PS_TTM,PC_TTM,PB,adjust_price_f,MA_5,EMA_5
0,sh600000,2014-12-31,15.45,15.79,15.11,15.69,0.021484,468832339.0,7.271525e+09,2.341384e+11,...,130.254642,2014-09-30,2014-10-31,6.375742,2.515326,3.401388,1.265111,15.081313,15.004,15.081567
1,sh600000,2014-12-30,14.95,15.50,14.83,15.36,0.027425,443655559.0,6.774519e+09,2.292139e+11,...,127.515107,2014-09-30,2014-10-31,6.241646,2.462423,3.329850,1.238503,14.764121,14.558,14.777350
2,sh600000,2014-12-29,15.40,15.88,14.71,14.95,0.012187,625216146.0,9.556436e+09,2.230955e+11,...,124.111353,2014-09-30,2014-10-31,6.075039,2.396694,3.240966,1.205444,14.370023,14.308,14.486025
3,sh600000,2014-12-26,14.30,14.84,14.19,14.77,0.036491,466983121.0,6.790353e+09,2.204094e+11,...,122.617020,2014-09-30,2014-10-31,6.001893,2.367837,3.201944,1.190930,14.197004,14.254,14.254038
4,sh600000,2014-12-25,13.75,14.27,13.53,14.25,0.058692,455721907.0,6.359588e+09,2.126496e+11,...,118.300130,2014-09-30,2014-10-31,5.790589,2.284474,3.089216,1.149001,13.697180,14.118,13.996056
